## 各种import

In [1]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("../examples"))

import utils; reload(utils)
from utils import *

from __future__ import division, print_function

Using TensorFlow backend.


## 从Keras导入数据

In [2]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## 处理数据

### One-Hot-Encoding

In [3]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道

In [4]:
trn_data = np.expand_dims(trn_data, 1)
test_data = np.expand_dims(test_data, 1)
trn_data.shape

(60000, 1, 28, 28)

### 对输入图像进行正则化

In [5]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

## 1. 简单神经网络模型

In [9]:
simpleNN = Sequential()
simpleNN.add(Lambda(norm_input, input_shape=(1, 28, 28)))
simpleNN.add(Flatten())
simpleNN.add(Dense(512, activation='softmax'))
simpleNN.add(Dense(10, activation='softmax'))
simpleNN.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

### 1.1 直接训练

In [10]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 20s - loss: 1.3982 - acc: 0.4827 - val_loss: 1.3680 - val_acc: 0.4840


In [8]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 16s - loss: 1.1765 - acc: 0.5399 - val_loss: 1.1229 - val_acc: 0.5472
Epoch 2/3
60000/60000 [==============================] - 16s - loss: 1.1632 - acc: 0.5477 - val_loss: 1.1065 - val_acc: 0.5672
Epoch 3/3
60000/60000 [==============================] - 15s - loss: 1.1738 - acc: 0.5511 - val_loss: 1.1937 - val_acc: 0.5570


### 1.2 生成batch再训练

利用ImageDataGenerator(), 使用fit_generator而不是fit
注意fit_generator的文档中参数由于版本升级发生了变化

In [11]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

In [12]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 18s - loss: 1.1761 - acc: 0.5554 - val_loss: 1.0962 - val_acc: 0.5897


In [54]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 20s - loss: 0.2186 - acc: 0.9448 - val_loss: 0.2648 - val_acc: 0.9351
Epoch 2/3
60000/60000 [==============================] - 23s - loss: 0.2065 - acc: 0.9476 - val_loss: 0.2312 - val_acc: 0.9435
Epoch 3/3
60000/60000 [==============================] - 23s - loss: 0.1947 - acc: 0.9511 - val_loss: 0.2390 - val_acc: 0.9396


## 2. 类似vgg16的卷积神经网络模型

In [83]:
simpleCNN = Sequential()
simpleCNN.add(Lambda(norm_input, input_shape=(1, 28, 28), output_shape=(1, 28, 28)))
simpleCNN.add(ZeroPadding2D())
simpleCNN.add(Convolution2D(64, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(ZeroPadding2D())
simpleCNN.add(Convolution2D(128, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(ZeroPadding2D())
simpleCNN.add(Convolution2D(256, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(Flatten())
simpleCNN.add(Dense(512, activation="softmax"))
simpleCNN.add(Dense(10, activation="softmax"))
simpleCNN.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

C:\anaconda2\lib\site-packages\keras\layers\core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_19 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [16]:
simpleCNN = Sequential()
simpleCNN.add(Lambda(norm_input, input_shape=(1, 28, 28), output_shape=(1, 28, 28)))
simpleCNN.add(Convolution2D(32, 3, 3, activation="relu"))
simpleCNN.add(Convolution2D(32, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(Convolution2D(64, 3, 3, activation="relu"))
simpleCNN.add(Convolution2D(64, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())

simpleCNN.add(Flatten())
simpleCNN.add(Dense(512, activation="relu"))
simpleCNN.add(Dense(10, activation="softmax"))
simpleCNN.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
simpleCNN.optimizer.lr = 0.1
simpleCNN.fit_generator(batches, 6000, nb_epoch=1, validation_data=test_batches, nb_val_samples=300)

Epoch 1/1
 512/6000 [=>............................] - ETA: 49s - loss: 13.1632 - acc: 0.0938

KeyboardInterrupt: 

In [18]:
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=64)
test_batches = gen.flow(test_data, test_label, batch_size=64)

In [19]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [20]:
model = get_model()
model.optimizer.lr = 0.1
model.fit_generator(batches, 1000, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=300)

Epoch 1/1
1024/1000 [==============================] - 9s - loss: 13.7599 - acc: 0.1006 - val_loss: 14.5567 - val_acc: 0.0969
